In [2]:
import pyedflib
import numpy as np
import matplotlib.pyplot as plt
import os


#Samples are represented in 16-bit 2's complement

# Get the directory where the script is located
script_dir = os.getcwd()
file_counter = 0
Files = []
badFiles = []
# Construct the full path to the file
file_path = os.path.join(script_dir, 'EDF', 'PD patient Frontal')
for filename in os.listdir(file_path):
    # Check if the file ends with .edf
    if filename.endswith('.edf'):
        file_counter = file_counter+1
        Files.append(filename)
for k in np.arange(file_counter):
    path = os.path.join(file_path, Files[k])
    try:
        f = pyedflib.EdfReader(path)
    except OSError:
        badFiles.append(Files[k])     
Files = [item for item in Files if item not in badFiles]
Files.reverse()
n = f.signals_in_file
n = n-9
number_of_samples = f.getNSamples()[0]
Nblocks = int((number_of_samples-250)/64)
TotalBlocks=(5*Nblocks)*n
fuller_data = np.ndarray(shape=(TotalBlocks, 64, 2))
BlockCount=0
multiplier = f.getPhysicalMaximum(0)/f.getDigitalMaximum(0)
f.close()
for index, name in enumerate(Files):
    print(name)
    path = os.path.join(file_path, name)
    f = pyedflib.EdfReader(path)
    number_of_samples = f.getNSamples()[0]
    Nblocks = int((number_of_samples-250)/64)
    sigbufs = np.zeros(number_of_samples)
    full_data64 = np.ndarray(shape=(Nblocks*n, 64, 2))
    signalList = []  
    BlockCount = BlockCount+Nblocks*n
    ran = np.ndarray(shape=(21, number_of_samples-250))
    for i in np.arange(n):
        sigbufs[:] = f.readSignal(i, digital=True)
        sigbufs_new = sigbufs[250:]
        #ran[i] = sigbufs_new
        signalList.append(sigbufs_new) 
        labels = np.zeros(number_of_samples-250)
        if name == "DP14.edf":
            sezStart = 79650
            sezEnd = 82250
            labels[sezStart:sezEnd] = 1
        elif name == "DP141.edf":
            sezStart = 103500
            sezEnd = 106500
            labels[sezStart:sezEnd] = 1
        elif name == "DP142.edf":
            sezStart = 223250
            sezEnd = 224750
            labels[sezStart:sezEnd] = 1
        elif name == "DP18.edf":
            sezStart = 93250
            sezEnd = 94000
            labels[sezStart:sezEnd] = 1
        for j in np.arange(Nblocks):
                full_data64[i*Nblocks+j,:,0] = signalList[i][j*64:(j+1)*64]
                full_data64[i*Nblocks+j,:,1] = labels[j*64:(j+1)*64]
    fuller_data[BlockCount-Nblocks*n:BlockCount] = full_data64
    f.close()
seizures = np.sum(fuller_data[:,:,1] == 1) 
normal = np.sum(fuller_data[:,:,1] == 0)
percentage_seizure = seizures/(seizures+normal)*100
print(fuller_data.shape)
print(percentage_seizure)

DP14.edf
DP18.edf
DP141.edf
DP142.edf
DP15.edf
(369495, 64, 2)
0.697108553566354


In [3]:
import tensorflow as tf
def prd_loss_dig2phy_new(y_true, y_pred):
    y_true_processed = y_true[:, :, 0]
    y_true_processed = y_true_processed[..., tf.newaxis]
    y_true_processed = (y_true_processed)*multiplier
    y_pred = (y_pred)*multiplier
    rms_deviation = (tf.reduce_sum(tf.square(y_true_processed - y_pred)))
    percentage_rmsd = tf.sqrt(rms_deviation/(tf.reduce_sum(tf.square(y_true_processed))+tf.keras.backend.epsilon()))* 100
    return percentage_rmsd

In [4]:
import tensorflow.keras.backend as K
def weighted_mse_loss(y_true, y_pred):
    # y_true, y_pred shape: (batch_size, num_channels, signal_length)
    # label shape: (batch_size, )
    weight = 150
    # Extract the labels (0 or 1) from the last dimension of y_true
    labels = y_true[:, :, 1]
    # Remove the labels from y_true for loss calculation
    y_true_processed = y_true[:, :, 0]
    y_true_processed=y_true_processed[..., tf.newaxis]
    loss = K.mean(K.square(y_pred - y_true_processed))
    weighted_loss = loss * ((labels * (weight - 1)) + 1)
    return K.mean(weighted_loss)

In [5]:
import tensorflow as tf
def prd_loss_dig2phy(y_true, y_pred):
    y_true = (y_true)*multiplier
    y_pred = (y_pred)*multiplier
    rms_deviation = (tf.reduce_sum(tf.square(y_true - y_pred)))
    percentage_rmsd = tf.sqrt(rms_deviation/(tf.reduce_sum(tf.square(y_true))+tf.keras.backend.epsilon()))* 100
    return percentage_rmsd

In [ ]:
from qkeras.utils import load_qmodel
from keras import layers, Sequential
Qencoder = load_qmodel('Qencoder.h5')
Qencoder.summary()
layer1 = Qencoder.get_layer('q_conv1d_1')
weights = layer1.get_weights()
print(weights)

In [5]:
import tensorflow as tf
from tensorflow import keras
from qkeras import quantizers
from qkeras import QConv1D, QDense, QActivation
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

# Define the encoder with quantization
input_ts = keras.layers.Input(shape=(64, 1), name="input_time_series") # 64, 1

# First Conv1D layer
x = QConv1D(8, 3, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, 1, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(15, 0, 1, alpha=1),
            activation='linear', 
            padding='same', )(input_ts) # 64, 16

# Second Conv1D layer
x = QConv1D(8, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, 1, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, 1, alpha=1),
            activation='linear', 
            padding='same')(x) # 64, 16

# Third Conv1D layer with strides
x = QConv1D(4, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, 1, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, 1, alpha=1),
            activation='linear', 
            padding='same', 
            strides=2)(x) # 32, 8

# Fourth Conv1D layer with strides
x = QConv1D(4, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, 1, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, 1, alpha=1),
            activation='linear', 
            padding='same', 
            strides=2)(x) # 16, 4

# Flatten layer
x = keras.layers.Flatten()(x) # Flatten for Dense layer

# Dense layer
encoded = QDense(16, 
                 kernel_quantizer=quantizers.quantized_bits(12, 0, 1, alpha=1), 
                 bias_quantizer=quantizers.quantized_bits(12, 0, 1, alpha=1),
                 activation='linear')(x) # 16

# Define the model
encoder = keras.models.Model(input_ts, encoded, name="encoder")

# Print the model summary
encoder.summary()

# Define the decoder
encoded_input = keras.layers.Input(shape=(16,), name="encoded_input") # 16
x = keras.layers.Dense(16 * 4, activation='linear')(encoded_input) # 16 * 4
x = keras.layers.Reshape((16, 4))(x) # Reshape back to (16, 4)
x = keras.layers.Conv1DTranspose(8, 5, activation='linear', strides=2, padding='same')(x) # 16, 8
x = keras.layers.Conv1DTranspose(16, 5, activation='linear', strides=2, padding='same')(x) # 32, 16
x = keras.layers.Conv1DTranspose(8, 7, activation='linear', padding='same')(x) # 32, 16
decoded = keras.layers.Conv1DTranspose(1, 3, activation='linear', padding='same')(x) # 64, 1

decoder = keras.models.Model(encoded_input, decoded, name="decoder")

# Define the autoencoder
autoencoder_input = keras.layers.Input(shape=(64, 1), name="autoencoder_input")
encoded_ts = encoder(autoencoder_input)
decoded_ts = decoder(encoded_ts)

autoencoder = keras.models.Model(autoencoder_input, decoded_ts, name="autoencoder")

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss=weighted_mse_loss, metrics=[prd_loss_dig2phy_new])

# Summary of the autoencoder
autoencoder.summary()
checkpoint_callback = ModelCheckpoint(
    filepath='quantised.h5',  # Path to save the model
    monitor='val_loss',        # Metric to monitor
    save_best_only=True,       # Save only the best model
    mode='min',                # Mode: minimize the monitored metric
    verbose=1                  # Print a message when saving the model
)
#features_train = fuller_data[:, :, 0]
# features_val = val_data[:, :, 0]

new_train, new_val = train_test_split(fuller_data, test_size=0.2, random_state=69)
features_val = new_val[:, :, 0]
features_val = features_val[..., tf.newaxis]
features_train = new_train[:, :, 0]
features_train = features_train[..., tf.newaxis]
#features_val = features_val[..., tf.newaxis]
#print(features_train.shape)
# Train the autoencoder
autoencoder.fit(features_train, new_train, epochs=30, shuffle=True, batch_size = 128, callbacks=[checkpoint_callback], validation_data=(features_val, new_val))



2024-08-05 15:07:15.694828: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-05 15:07:15.887540: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-05 15:07:15.887766: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-05 15:07:15.890228: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-05 15:07:15.890437: I external/local_xla/xla/stream_executor

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_time_series (InputLa  [(None, 64, 1)]           0         
 yer)                                                            
                                                                 
 q_conv1d (QConv1D)          (None, 64, 8)             32        
                                                                 
 q_conv1d_1 (QConv1D)        (None, 64, 8)             328       
                                                                 
 q_conv1d_2 (QConv1D)        (None, 32, 4)             164       
                                                                 
 q_conv1d_3 (QConv1D)        (None, 16, 4)             84        
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                           

2024-08-05 15:07:25.777937: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-08-05 15:07:27.689071: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-05 15:07:29.622295: I external/local_xla/xla/service/service.cc:168] XLA service 0x4625d840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-05 15:07:29.622341: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2024-08-05 15:07:29.643896: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1722866849.747892  361351 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2310/2310 [==============================] - ETA: 0s - loss: 49440.7070 - prd_loss_dig2phy_new: 52.4926
Epoch 1: val_loss improved from inf to 36747.25391, saving model to quantised.h5
2310/2310 [==============================] - 48s 16ms/step - loss: 49440.7070 - prd_loss_dig2phy_new: 52.4926 - val_loss: 36747.2539 - val_prd_loss_dig2phy_new: 37.3019
Epoch 2/30
  11/2310 [..............................] - ETA: 25s - loss: 31692.8516 - prd_loss_dig2phy_new: 32.6840

/home/berkb/miniconda3/envs/qkeras/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/home/berkb/miniconda3/envs/qkeras/lib/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


2310/2310 [==============================] - ETA: 0s - loss: 28029.5762 - prd_loss_dig2phy_new: 35.9826
Epoch 2: val_loss improved from 36747.25391 to 26263.85352, saving model to quantised.h5
2310/2310 [==============================] - 36s 16ms/step - loss: 28029.5762 - prd_loss_dig2phy_new: 35.9826 - val_loss: 26263.8535 - val_prd_loss_dig2phy_new: 29.6533
Epoch 3/30
2309/2310 [============================>.] - ETA: 0s - loss: 20964.1914 - prd_loss_dig2phy_new: 33.5341
Epoch 3: val_loss did not improve from 26263.85352
2310/2310 [==============================] - 36s 16ms/step - loss: 20963.5215 - prd_loss_dig2phy_new: 33.5464 - val_loss: 67015.1719 - val_prd_loss_dig2phy_new: 61.9282
Epoch 4/30
2309/2310 [============================>.] - ETA: 0s - loss: 17776.8711 - prd_loss_dig2phy_new: 30.4397
Epoch 4: val_loss improved from 26263.85352 to 21695.84570, saving model to quantised.h5
2310/2310 [==============================] - 46s 20ms/step - loss: 17774.2852 - prd_loss_dig2phy_ne

In [10]:
import tensorflow as tf
from tensorflow import keras
from qkeras import quantizers
from qkeras import QConv1D, QDense, QActivation
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from qkeras import quantizers
from qkeras import QConv1D, QDense, QActivation
from qkeras.utils import load_qmodel
from keras.optimizers import Adam

# Define the encoder with quantization
input_ts = keras.layers.Input(shape=(64, 1), name="input_time_series")  # 64, 1

# First Conv1D layer with quantized activations
x = QConv1D(8, 3, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same')(input_ts)  # 64, 16
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)  # Quantized activations to 16 bits
# Second Conv1D layer with quantized activations
x = QConv1D(8, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same')(x)  # 64, 16
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)
# Third Conv1D layer with quantized activations
x = QConv1D(4, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same', 
            strides=2)(x)  # 32, 8
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)
# Fourth Conv1D layer with quantized activations
x = QConv1D(4, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same', 
            strides=2)(x)  # 16, 4
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)
# Flatten layer
x = keras.layers.Flatten()(x)  # Flatten for Dense layer

# Dense layer with quantized activations
x = QDense(16, kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
                 bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1))(x)  # 16
encoded = QActivation(activation= quantizers.quantized_bits(16, 15, alpha=1))(x)

# Define the model
encoder = keras.models.Model(input_ts, encoded, name="encoder")

# Print the model summary
encoder.summary()

# Define the decoder
encoded_input = keras.layers.Input(shape=(16,), name="encoded_input") # 16
x = keras.layers.Dense(16 * 4, activation='linear')(encoded_input) # 16 * 4
x = keras.layers.Reshape((16, 4))(x) # Reshape back to (16, 4)
x = keras.layers.Conv1DTranspose(8, 5, activation='linear', strides=2, padding='same')(x) # 16, 8
x = keras.layers.Conv1DTranspose(16, 5, activation='linear', strides=2, padding='same')(x) # 32, 16
x = keras.layers.Conv1DTranspose(8, 7, activation='linear', padding='same')(x) # 32, 16
decoded = keras.layers.Conv1DTranspose(1, 3, activation='linear', padding='same')(x) # 64, 1

decoder = keras.models.Model(encoded_input, decoded, name="decoder")

# Define the autoencoder
autoencoder_input = keras.layers.Input(shape=(64, 1), name="autoencoder_input")
encoded_ts = encoder(autoencoder_input)
decoded_ts = decoder(encoded_ts)

autoencoder = keras.models.Model(autoencoder_input, decoded_ts, name="autoencoder")

# Compile the autoencoder
autoencoder.compile(optimizer= Adam(learning_rate=0.0001), loss=weighted_mse_loss, metrics=[prd_loss_dig2phy_new])
weight_model = load_qmodel('inter1_quantised.h5', custom_objects={'prd_loss_dig2phy_new': prd_loss_dig2phy_new, 'weighted_mse_loss': weighted_mse_loss})
# Summary of the autoencoder
autoencoder.summary()
autoencoder.set_weights(weight_model.get_weights())
checkpoint_callback = ModelCheckpoint(
    filepath='real_quantised.h5',  # Path to save the model
    monitor='val_loss',        # Metric to monitor
    save_best_only=True,       # Save only the best model
    mode='min',                # Mode: minimize the monitored metric
    verbose=1                  # Print a message when saving the model
)
new_train, new_val = train_test_split(fuller_data, test_size=0.2, random_state=69)
features_val = new_val[:, :, 0]
features_val = features_val[..., tf.newaxis]
features_train = new_train[:, :, 0]
features_train = features_train[..., tf.newaxis]
#pred = autoencoder.predict(features_val)
#print(prd_loss_dig2phy(features_val, pred).numpy())
# Train the autoencoder
autoencoder.fit(features_train, new_train, epochs=30, batch_size=128, shuffle=True, callbacks=[checkpoint_callback], validation_data=(features_val, new_val))


Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_time_series (InputLa  [(None, 64, 1)]           0         
 yer)                                                            
                                                                 
 q_conv1d_16 (QConv1D)       (None, 64, 8)             32        
                                                                 
 q_activation_20 (QActivati  (None, 64, 8)             0         
 on)                                                             
                                                                 
 q_conv1d_17 (QConv1D)       (None, 64, 8)             328       
                                                                 
 q_activation_21 (QActivati  (None, 64, 8)             0         
 on)                                                             
                                                           

/home/berkb/miniconda3/envs/qkeras/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/home/berkb/miniconda3/envs/qkeras/lib/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


2308/2310 [============================>.] - ETA: 0s - loss: 10307.5283 - prd_loss_dig2phy_new: 21.7544
Epoch 2: val_loss did not improve from 16925.25977
2310/2310 [==============================] - 44s 19ms/step - loss: 10301.9971 - prd_loss_dig2phy_new: 21.7520 - val_loss: 18517.0078 - val_prd_loss_dig2phy_new: 22.1772
Epoch 3/30
2309/2310 [============================>.] - ETA: 0s - loss: 13783.1143 - prd_loss_dig2phy_new: 24.9550
Epoch 3: val_loss did not improve from 16925.25977
2310/2310 [==============================] - 41s 18ms/step - loss: 13781.1221 - prd_loss_dig2phy_new: 24.9516 - val_loss: 24855.1035 - val_prd_loss_dig2phy_new: 26.0663
Epoch 4/30
2306/2310 [============================>.] - ETA: 0s - loss: 26885.7715 - prd_loss_dig2phy_new: 40.0140
Epoch 4: val_loss did not improve from 16925.25977
2310/2310 [==============================] - 38s 17ms/step - loss: 26851.0586 - prd_loss_dig2phy_new: 40.0287 - val_loss: 58607.4297 - val_prd_loss_dig2phy_new: 50.0580
Epoch 

KeyboardInterrupt: 

In [7]:
import tensorflow as tf
from tensorflow import keras
from qkeras import quantizers
from qkeras import QConv1D, QDense, QActivation
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from qkeras import quantizers
from qkeras import QConv1D, QDense, QActivation
from qkeras.utils import load_qmodel
from keras.optimizers import Adam

# Define the encoder with quantization
input_ts = keras.layers.Input(shape=(64, 1), name="input_time_series")  # 64, 1

# First Conv1D layer with quantized activations
x = QConv1D(8, 3, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same')(input_ts)  # 64, 16
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)  # Quantized activations to 16 bits
# Second Conv1D layer with quantized activations
x = QConv1D(8, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same')(x)  # 64, 16
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)
# Third Conv1D layer with quantized activations
x = QConv1D(4, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same', 
            strides=2)(x)  # 32, 8
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)
# Fourth Conv1D layer with quantized activations
x = QConv1D(4, 5, 
            kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
            bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1),
            padding='same', 
            strides=2)(x)  # 16, 4
x = QActivation(activation= quantizers.quantized_bits(28, 15, alpha=1))(x)
# Flatten layer
x = keras.layers.Flatten()(x)  # Flatten for Dense layer

# Dense layer with quantized activations
x = QDense(16, kernel_quantizer=quantizers.quantized_bits(12, 0, alpha=1), 
                 bias_quantizer=quantizers.quantized_bits(12, 0, alpha=1))(x)  # 16
encoded = keras.layers.Activation(activation='linear')(x)

# Define the model
encoder = keras.models.Model(input_ts, encoded, name="encoder")

# Print the model summary
encoder.summary()

# Define the decoder
encoded_input = keras.layers.Input(shape=(16,), name="encoded_input") # 16
x = keras.layers.Dense(16 * 4, activation='linear')(encoded_input) # 16 * 4
x = keras.layers.Reshape((16, 4))(x) # Reshape back to (16, 4)
x = keras.layers.Conv1DTranspose(8, 5, activation='linear', strides=2, padding='same')(x) # 16, 8
x = keras.layers.Conv1DTranspose(16, 5, activation='linear', strides=2, padding='same')(x) # 32, 16
x = keras.layers.Conv1DTranspose(8, 7, activation='linear', padding='same')(x) # 32, 16
decoded = keras.layers.Conv1DTranspose(1, 3, activation='linear', padding='same')(x) # 64, 1

decoder = keras.models.Model(encoded_input, decoded, name="decoder")

# Define the autoencoder
autoencoder_input = keras.layers.Input(shape=(64, 1), name="autoencoder_input")
encoded_ts = encoder(autoencoder_input)
decoded_ts = decoder(encoded_ts)

autoencoder = keras.models.Model(autoencoder_input, decoded_ts, name="autoencoder")

# Compile the autoencoder
autoencoder.compile(optimizer=Adam(learning_rate=0.00001), loss=weighted_mse_loss, metrics=[prd_loss_dig2phy_new])
weight_model = load_qmodel('quantised.h5', custom_objects={'prd_loss_dig2phy_new': prd_loss_dig2phy_new, 'weighted_mse_loss': weighted_mse_loss})
# Summary of the autoencoder
autoencoder.summary()
autoencoder.set_weights(weight_model.get_weights())
checkpoint_callback = ModelCheckpoint(
    filepath='inter1_quantised.h5',  # Path to save the model
    monitor='val_loss',        # Metric to monitor
    save_best_only=True,       # Save only the best model
    mode='min',                # Mode: minimize the monitored metric
    verbose=1                  # Print a message when saving the model
)
#features_train = fuller_data[:, :, 0]
# features_val = val_data[:, :, 0]

new_train, new_val = train_test_split(fuller_data, test_size=0.2, random_state=69)
features_val = new_val[:, :, 0]
features_val = features_val[..., tf.newaxis]
features_train = new_train[:, :, 0]
features_train = features_train[..., tf.newaxis]
pred = autoencoder.predict(features_val)
print(prd_loss_dig2phy(features_val, pred).numpy())
# Train the autoencoder
autoencoder.fit(features_train, new_train, epochs=30, batch_size=128, shuffle=True, callbacks=[checkpoint_callback], validation_data=(features_val, new_val))



Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_time_series (InputLa  [(None, 64, 1)]           0         
 yer)                                                            
                                                                 
 q_conv1d_8 (QConv1D)        (None, 64, 8)             32        
                                                                 
 q_activation_4 (QActivatio  (None, 64, 8)             0         
 n)                                                              
                                                                 
 q_conv1d_9 (QConv1D)        (None, 64, 8)             328       
                                                                 
 q_activation_5 (QActivatio  (None, 64, 8)             0         
 n)                                                              
                                                           

/home/berkb/miniconda3/envs/qkeras/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/home/berkb/miniconda3/envs/qkeras/lib/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


2308/2310 [============================>.] - ETA: 0s - loss: 7207.9482 - prd_loss_dig2phy_new: 20.4545
Epoch 2: val_loss improved from 14678.71191 to 14632.60742, saving model to inter1_quantised.h5
2310/2310 [==============================] - 39s 17ms/step - loss: 7222.4810 - prd_loss_dig2phy_new: 20.4522 - val_loss: 14632.6074 - val_prd_loss_dig2phy_new: 21.3138
Epoch 3/30
2309/2310 [============================>.] - ETA: 0s - loss: 8719.5635 - prd_loss_dig2phy_new: 20.3892
Epoch 3: val_loss improved from 14632.60742 to 14615.87012, saving model to inter1_quantised.h5
2310/2310 [==============================] - 45s 20ms/step - loss: 8718.4561 - prd_loss_dig2phy_new: 20.3864 - val_loss: 14615.8701 - val_prd_loss_dig2phy_new: 21.2842
Epoch 4/30
2306/2310 [============================>.] - ETA: 0s - loss: 7311.6304 - prd_loss_dig2phy_new: 20.3505
Epoch 4: val_loss improved from 14615.87012 to 14607.24414, saving model to inter1_quantised.h5
2310/2310 [==============================] - 

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import qkeras
from qkeras.utils import load_qmodel
from sklearn.model_selection import train_test_split    
new_train, new_val = train_test_split(fuller_data, test_size=0.2, random_state=69)
features_val = new_val[:, :, 0]
features_val = features_val[..., tf.newaxis]
features_train = new_train[:, :, 0]
features_train = features_train[..., tf.newaxis]
weight_model = load_qmodel('inter_quantised.h5', custom_objects={'prd_loss_dig2phy_new': prd_loss_dig2phy_new, 'weighted_mse_loss': weighted_mse_loss})
lets = load_qmodel('real1_quantised.h5', custom_objects={'prd_loss_dig2phy_new': prd_loss_dig2phy_new, 'weighted_mse_loss': weighted_mse_loss})
pred = lets.predict(features_val)
print(prd_loss_dig2phy(features_val, pred).numpy())
lets.set_weights(weight_model.get_weights())
pred = lets.predict(features_val)
print(prd_loss_dig2phy(features_val, pred).numpy())

2310/2310 [==============================] - 13s 6ms/step
36.401436645508674
2310/2310 [==============================] - 15s 6ms/step
36.314028973267845
